In [ ]:
import numpy as np
import pandas as pd



In [ ]:
## load data
final_result = pd.read_csv('processed_incident_count_005.csv')

df = pd.DataFrame()
for i in range(505):
    path = f"/Users/pei/Desktop/Pei/Research/RA role in University/utd_prof_ding/zDPD_data_0-10999999/zDPD_data_{1999+i*2000}.csv"
    df = pd.concat([df, pd.read_csv(path).iloc[:,1:]])
print(df.shape)
df.head(1)

In [ ]:
df['lon_bin'] = df['lon_bin'].round(2)
df['lat_bin'] = df['lat_bin'].round(2)
final_result['date'] = final_result['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df = df.merge(final_result, how = "left", 
              left_on = ['lon_bin', 'lat_bin', 'date'], right_on = ['lon_bin', 'lat_bin', 'date'],
              suffixes=['_drop', ''])
df.head(1)

In [ ]:
import xgboost as xgb
# import pandas as pd
# import numpy as np
from sklearn.metrics import root_mean_squared_error

# Ensure the data is sorted by date
df = df.sort_values(by='date')
# Assuming df['date'] contains datetime objects
df['date'] = pd.to_datetime(df['date'])

# Extract year, day of year, week of year, and month-day components
df['date_year'] = df['date'].dt.year
df['day_of_year'] = df['date'].dt.dayofyear
df['week_of_year'] = df['date'].dt.isocalendar().week

# Encode day_of_year cyclically
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

# Encode week_of_year cyclically
df['week_of_year_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
df['week_of_year_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)

In [ ]:
# Define features and target
features = ['lon_bin', 'lat_bin', 'date_year', 'day_of_year_sin', 'day_of_year_cos', 'week_of_year_sin', 'week_of_year_cos',
            'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'Unemployment_Rate(%)', 'Unemployment_Rank', 'CPI(Annual)']
target = 'unique_event_count'

# Train-test split
train_data = df[df['date'] < '2024-01-01']  # Train on data before 2023
test_data = df[df['date'] >= '2024-01-01']  # Test on data from 2023 onward

# Convert training data to DMatrix for XGBoost
dtrain = xgb.DMatrix(train_data[features], label=train_data[target])


# poisson_obj for event count prediction assuming the count follow poisson distribution
def poisson_obj(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.exp(preds)  # Ensure predictions are positive
    grad = preds - labels  # Gradient: difference between prediction and true count
    hess = preds           # Hessian: prediction values (positive)
    return grad, hess


# Set XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'eta': 0.01,  # Learning rate
    'eval_metric': 'rmse',
    'scale_pos_weight': 10
}


# Train initial model
model = xgb.train(params, dtrain, num_boost_round=100, obj=poisson_obj)

# Group test data by date
test_dates = sorted(test_data['date'].unique())  # Get unique dates in the test set

# Rolling evaluation with incremental updates
errors = []  # To store RMSE for each day
predictions = []
for test_date in test_dates:
    # Get all rows for the current test date
    test_day = test_data[test_data['date'] == test_date]
    
    # Create a DMatrix for the test day's data
    dtest = xgb.DMatrix(test_day[features])
    y_test = test_day[target].values  # True values for all locations on the current day
    
    # Predict for the test day's data
    y_pred = model.predict(dtest)
    
    # Transform predictions back to original scale
    y_pred_exp = np.exp(y_pred)  # Apply exponential transformation
    
    # Enforce integer predictions
    y_pred_rounded = np.round(y_pred_exp).astype(int)
    
    # Ensure non-negative predictions
    y_pred_rounded = np.clip(y_pred_rounded, 0, None)

    # Calculate RMSE for the day
    rmse = np.sqrt(np.mean((y_test - y_pred_rounded) ** 2))
    errors.append(rmse)
    predictions.extend(y_pred_rounded)
    
    # Update the model with the test day's data
    dnew = xgb.DMatrix(test_day[features], label=test_day[target])
    model = xgb.train(params, xgb.DMatrix(pd.concat([train_data[features], test_day[features]]), 
                                          label=pd.concat([train_data[target], test_day[target]])),
                      obj=poisson_obj)
    
    # Optional: Add the test day's data to the training dataset
    train_data = pd.concat([train_data, test_day])

# Overall evaluation
overall_rmse = np.mean(errors)
print(f"Overall RMSE: {overall_rmse}")